<a href="https://colab.research.google.com/github/UmairPirzada/Crewai-Research-Agents-Reporting-Analyst/blob/main/Research_write_article_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: The Future of Artificial Intelligence: Latest Trends and Key Players

I. Introduction
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention the exponential growth and advancements in AI technology

II. Latest Trends in Artificial Intelligence
A. Machine Learning
- Discuss the importance of machine learning in AI development
- Highlight recent breakthroughs in machine learning algorithms

B. Natural Language Processing (NLP)
- Explain the significance of NLP in A

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Latest Trends and Key Players

## Introduction
Artificial Intelligence (AI) has revolutionized various industries with its innovative technologies. The exponential growth and advancements in AI have paved the way for groundbreaking developments that are reshaping the future of technology. With the increasing integration of AI into everyday life, it is crucial to stay informed about the latest trends and key players in the industry to understand its impact on society and businesses.

## Latest Trends in Artificial Intelligence

### Machine Learning
Machine learning plays a crucial role in the development of AI by enabling systems to learn and improve from experience. Recent breakthroughs in machine learning algorithms have led to significant advancements in AI capabilities, such as predictive analytics and deep learning models. The ability of machines to process and analyze vast amounts of data has rev

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends and Key Players

## Introduction
Artificial Intelligence (AI) has revolutionized various industries with its innovative technologies. The exponential growth and advancements in AI have paved the way for groundbreaking developments that are reshaping the future of technology. With the increasing integration of AI into everyday life, it is crucial to stay informed about the latest trends and key players in the industry to understand its impact on society and businesses.

## Latest Trends in Artificial Intelligence

### Machine Learning
Machine learning plays a crucial role in the development of AI by enabling systems to learn and improve from experience. Recent breakthroughs in machine learning algorithms have led to significant advancements in AI capabilities, such as predictive analytics and deep learning models. The ability of machines to process and analyze vast amounts of data has revolutionized industries like healthcare, finance, and marketing, driving efficiency and innovation.

### Natural Language Processing (NLP)
NLP is essential in AI applications as it allows machines to understand, interpret, and generate human language. Advancements in NLP have led to the widespread use of chatbots and virtual assistants, enhancing customer interactions and user experiences. By leveraging NLP technologies, businesses can automate customer service, streamline communication processes, and personalize user interactions, ultimately improving customer satisfaction and loyalty.

### Computer Vision
Computer vision is at the forefront of AI technology, enabling machines to interpret and understand visual information. Recent developments in image recognition and object detection have revolutionized industries such as autonomous vehicles, healthcare imaging, and security systems. The integration of computer vision technologies has led to enhanced safety measures, improved diagnostic capabilities, and increased operational efficiency in various sectors, showcasing the transformative potential of AI in visual data analysis.

## Key Players in the AI Industry

### Google
Google is a key player in the AI industry, with numerous initiatives and projects focused on advancing AI technology. Their research and development efforts have led to significant breakthroughs in areas such as natural language understanding, image recognition, and self-learning algorithms. By investing in AI research and innovation, Google continues to drive the evolution of AI technologies and their applications across different sectors, shaping the future of intelligent systems and services.

### Microsoft
Microsoft is at the forefront of AI technologies, offering solutions that empower businesses and individuals to harness the power of AI. Their contributions to the AI industry include innovations in machine learning, cognitive services, and AI-driven productivity tools. With a strong commitment to democratizing AI and making it accessible to a wider audience, Microsoft is driving AI adoption and integration in diverse fields, fostering innovation and growth in the digital economy.

### IBM
IBM is a leader in AI innovation and research, with a strong focus on developing AI products and services that drive business growth. Their AI initiatives span across industries such as healthcare, finance, and cybersecurity, showcasing the transformative potential of AI technologies. By collaborating with industry partners and leveraging their expertise in AI research, IBM continues to push the boundaries of AI innovation, creating solutions that address complex challenges and drive sustainable business outcomes.

## Noteworthy News in Artificial Intelligence

### Ethical AI
Ethical considerations are crucial in AI development to ensure the responsible and fair use of AI technologies. Recent news on ethical AI practices and regulations highlight the importance of transparency, accountability, and bias mitigation in AI systems. By promoting ethical AI standards and guidelines, organizations can build trust with users and stakeholders, fostering a culture of responsible AI development and deployment in the digital age.

### AI in Healthcare
AI has had a profound impact on healthcare by improving medical diagnostics, patient care, and treatment outcomes. Recent advancements in AI-powered healthcare solutions, such as predictive analytics, personalized medicine, and telehealth services, have revolutionized the healthcare industry. By harnessing the power of AI technologies, healthcare providers can enhance clinical decision-making, optimize treatment protocols, and deliver personalized care to patients, ultimately improving health outcomes and quality of life.

## Target Audience Analysis
Our target audience includes tech-savvy professionals, entrepreneurs, and students interested in AI trends and developments. By providing insights into the latest AI trends, key industry players, and ethical AI practices, we aim to address their interests and pain points in understanding complex AI concepts and staying updated on AI news. Through informative and engaging content, we strive to empower our audience with knowledge and resources to navigate the evolving landscape of Artificial Intelligence and make informed decisions in their personal and professional lives.

## Call to Action
Stay informed about the latest trends and advancements in Artificial Intelligence by following our blog for regular updates and insights. Join our community of AI enthusiasts and industry professionals to engage in discussions, share insights, and collaborate on innovative AI projects. Together, we can shape the future of AI and drive positive change in the world through responsible and ethical AI practices. Keep exploring, learning, and innovating with us as we journey into the exciting realm of Artificial Intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "Data science"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: Exploring the Latest Trends in Technology

Introduction: 
- Brief overview of the rapid advancements in technology
- Importance of staying updated with the latest trends

Key Points: 
1. Latest Trends in Technology
- Artificial Intelligence and Machine Learning
- Internet of Things (IoT)
- Virtual Reality and Augmented Reality
- Blockchain Technology
- Cybersecurity

2. Key Players in the Industry
- Google
- Apple
- Microsoft
- Amazon
- Tesla

3. Noteworthy News in Technology
- Recent product 

I now can give a great answer

Final Answer: 

# Exploring the Latest Trends in Technology

In today's fast-paced world, technology is constantly evolving, bringing about new innovations and advancements at an unprecedented rate. It is crucial for individuals and businesses alike to stay updated with the latest trends in technology to remain competitive and relevant in their respective industries. From artificial intelligence and machine learning to blockchain technology and cybersecurity, the tech landscape is constantly changing, presenting both opportunities and challenges for those who seek to harness its potential.

## Latest Trends in Technology

One of the most prominent trends in technology today is the rise of artificial intelligence (AI) and machine learning. These technologies have revolutionized various industries, from healthcare and finance to marketing and transportation. The Internet of Things (IoT) has also gained significant traction, connecting devices and enabling s

In [ ]:
Markdown(result)

# Exploring the Latest Trends in Technology

In today's fast-paced world, technology is constantly evolving, bringing about new innovations and advancements at an unprecedented rate. It is crucial for individuals and businesses alike to stay updated with the latest trends in technology to remain competitive and relevant in their respective industries. From artificial intelligence and machine learning to blockchain technology and cybersecurity, the tech landscape is constantly changing, presenting both opportunities and challenges for those who seek to harness its potential.

## Latest Trends in Technology

One of the most prominent trends in technology today is the rise of artificial intelligence (AI) and machine learning. These technologies have revolutionized various industries, from healthcare and finance to marketing and transportation. The Internet of Things (IoT) has also gained significant traction, connecting devices and enabling seamless communication between them. Virtual reality (VR) and augmented reality (AR) are transforming the way we experience the world around us, while blockchain technology is reshaping the way we conduct transactions and secure data. Cybersecurity remains a top priority for businesses and individuals alike, as the threat of cyber attacks continues to loom large.

## Key Players in the Industry

When it comes to the tech industry, certain players stand out for their innovation and influence. Companies like Google, Apple, Microsoft, Amazon, and Tesla are at the forefront of technological advancements, constantly pushing the boundaries of what is possible. Whether it's developing cutting-edge products or investing in groundbreaking research, these companies play a crucial role in shaping the future of technology and driving progress in the industry.

## Noteworthy News in Technology

Keeping up with the latest news and developments in the tech sector is essential for staying informed and making informed decisions. Whether it's recent product launches, mergers and acquisitions, or market trends and forecasts, staying abreast of these developments can provide valuable insights into the direction of the industry. Resources like TechCrunch, Wired, Forbes Technology, MIT Technology Review, and Gartner Research offer a wealth of information on the latest trends and updates in technology, helping readers stay ahead of the curve.

In conclusion, the blog article on exploring the latest trends in technology aims to provide valuable insights and practical tips for tech enthusiasts, IT professionals, small business owners, and students looking to navigate the ever-changing tech landscape. By staying informed and leveraging the power of technology, individuals and businesses can unlock new opportunities and stay ahead in the fast-paced tech industry.